# 导入packages

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
import os
import html
import matplotlib.pyplot as plt
import seaborn as sb

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, Phrases
from sklearn.model_selection import train_test_split


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack # 水平堆叠稀疏矩阵(按列)
from sklearn.feature_extraction.text import TfidfVectorizer #把原始文本转化为tf-idf的特征矩阵，文本相似度计算
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV


xgb = XGBClassifier()
lr = LogisticRegression()
r = RandomForestClassifier()



# other_params = {'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
#                     'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

# param_test1 = {
#  'max_depth':list(range(2,6,9)),
#  'min_child_weight':list(range(2,4,6))
# }

# xgb1 = XGBClassifier(**other_params)
# gs1 = GridSearchCV(xgb1,param_test1,cv = 5,scoring = 'roc_auc',n_jobs = -1,verbose=2)
# best_model1=gs1.fit(x_train.values, y_train.values.reshape(-1))
# print('最优参数：',best_model1.best_params_)
# print('最佳模型得分：',best_model1.best_score_)

# 导入数据：数据探索性分析

In [40]:
# 本地路径
train=pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/labeledTrainData.tsv",'\t', error_bad_lines= False)
test=pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/testData.tsv","\t", error_bad_lines=False)
unlabeled = pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/unlabeledTrainData.tsv", "\t", error_bad_lines=False)
# 在线路径 ../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
# train=pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip",'\t', error_bad_lines= False)
# test=pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip","\t", error_bad_lines=False)
# unlabeled = pd.read_csv("../input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip", "\t", error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [4]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [6]:
unlabeled

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."
...,...,...
49993,18984_0,The original Man Eater by Joe D'Amato is somew...
49994,16433_0,When Home Box Office was in it's early days mo...
49995,16006_0,Griffin Dunne was born into a cultural family....
49996,40155_0,"Not a bad story, but the low budget rears its ..."


In [7]:
# 本地路径
# att=pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/imdb_master.csv", error_bad_lines=False)
# 在线路径
# att=pd.read_csv("../input/imdb-review-dataset/imdb_master.csv", error_bad_lines=False, encoding = 'utf-8', engine = 'python')
# att.head()

In [8]:
# import chardet
# with open("../input/imdb-review-dataset/imdb_master.csv", 'rb') as f:
#     result = chardet.detect(f.read())
#     print(result['encoding'])

In [9]:
# att=att[["review","label"]]
# att.columns=["review","sentiment"]
# att=att[att.sentiment != "unsup"]
# att=att.replace({"neg":0,"pos":1})
# att.head()

In [10]:
train.shape, test.shape, unlabeled.shape

((25000, 3), (25000, 2), (49998, 2))

In [11]:
# df=train.append(att, ignore_index=True)
df=train.append(test, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         50000 non-null  object 
 1   sentiment  25000 non-null  float64
 2   review     50000 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.1+ MB


In [12]:
df.head()

,id,sentiment,review
0,5814_8,1.0,With all this stuff going down at the moment w...
1,2381_9,1.0,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0.0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0.0,It must be assumed that those who praised this...
4,9495_8,1.0,Superbly trashy and wondrously unpretentious 8...


# 数据清洗

In [13]:
df.review=html.unescape(df.review)
# 去除网页标记

df.review=df.review.str.replace('http\S+|www.\S+', '', case=False).str.replace(r"\&\#[0-9]+\;","", regex=True) \
            .str.replace(r"[^\w\s]","").str.replace("\d+","").str.replace(r"[\s]+"," ",regex=True) \
            .str.replace("\n"," ").replace("\r","").str.replace("_"," ").str.lower()

In [14]:
def remove_emoji(text):
    '''使用正则表达式去除网页标记'''
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" 
                           u"\U0001F300-\U0001F5FF" 
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['review']=df['review'].apply(lambda x: remove_emoji(x))

In [15]:
from unicodedata import normalize

remove_accent = lambda text: normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore") # 字符串标准化

df["review"] = df["review"].apply(remove_accent)

In [16]:
df.head()

,id,sentiment,review
0,5814_8,1.0,with all this stuff going down at the moment w...
1,2381_9,1.0,the classic war of the worlds by timothy hines...
2,7759_3,0.0,the film starts with a manager nicholas bell g...
3,3630_4,0.0,it must be assumed that those who praised this...
4,9495_8,1.0,superbly trashy and wondrously unpretentious s...


In [17]:
df.sentiment.value_counts()

1.0    12500
0.0    12500
Name: sentiment, dtype: int64

# 使用review建语料库，并分词

In [18]:
%%time
corpora = df["review"].values
tokenized = [word_tokenize(corpus) for corpus in corpora]

print(tokenized[2222])

['go', 'immediately', 'and', 'rent', 'this', 'movie', 'it', 'will', 'be', 'be', 'on', 'a', 'bottom', 'shelf', 'in', 'your', 'local', 'video', 'store', 'and', 'will', 'be', 'covered', 'in', 'dust', 'no', 'one', 'will', 'have', 'touched', 'it', 'in', 'years', 'it', 'may', 'even', 'be', 'a', 'special', 'its', 'worth', 'ten', 'bucks', 'i', 'swear', 'buy', 'it', 'there', 'arent', 'very', 'many', 'films', 'than', 'can', 'compare', 'with', 'this', 'the', 'celluloid', 'version', 'of', 'that', 'goo', 'that', 'forms', 'at', 'the', 'bottom', 'of', 'a', 'trash', 'can', 'after', 'a', 'few', 'years', 'yes', 'i', 'gave', 'it', 'a', 'but', 'it', 'really', 'deserves', 'much', 'lower', 'scales', 'were', 'not', 'designed', 'with', 'stuff', 'like', 'this', 'in', 'mind']
Wall time: 35.5 s


In [19]:
%%time
tokenized = [list(filter(lambda x: len(x) > 1, document)) \
             for document in tokenized] 
# filter() 函数用于过滤序列，过滤掉不符合条件的元素，返回由符合条件元素组成的新列表。
print(tokenized[2222])

['go', 'immediately', 'and', 'rent', 'this', 'movie', 'it', 'will', 'be', 'be', 'on', 'bottom', 'shelf', 'in', 'your', 'local', 'video', 'store', 'and', 'will', 'be', 'covered', 'in', 'dust', 'no', 'one', 'will', 'have', 'touched', 'it', 'in', 'years', 'it', 'may', 'even', 'be', 'special', 'its', 'worth', 'ten', 'bucks', 'swear', 'buy', 'it', 'there', 'arent', 'very', 'many', 'films', 'than', 'can', 'compare', 'with', 'this', 'the', 'celluloid', 'version', 'of', 'that', 'goo', 'that', 'forms', 'at', 'the', 'bottom', 'of', 'trash', 'can', 'after', 'few', 'years', 'yes', 'gave', 'it', 'but', 'it', 'really', 'deserves', 'much', 'lower', 'scales', 'were', 'not', 'designed', 'with', 'stuff', 'like', 'this', 'in', 'mind']
Wall time: 1.95 s


# Word Embedding for Feature Engineering

In [20]:
%%time
from gensim.models import word2vec
np.set_printoptions(suppress=True)

feature_size = 256
context_size = 5
min_word = 1

word_vec= word2vec.Word2Vec(tokenized, vector_size=feature_size, \
                            window=context_size, min_count=min_word, \
                            epochs=50, seed=42)

Wall time: 8min 23s


# 获得最终w2v的数据表

In [21]:
%%time
word_vec_unpack = [(word, idx) for word, idx in \
                   word_vec.wv.key_to_index.items()]

tokens, indexes = zip(*word_vec_unpack)

word_vec_df = pd.DataFrame(word_vec.wv.vectors[indexes, :], index=tokens)

word_vec_df.head()

Wall time: 675 ms


,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
the,3.098315,2.621972,0.348336,-1.553718,-2.121981,-0.038903,-0.451384,0.784318,1.352593,0.375178,...,-0.852745,-0.491608,-1.617278,0.774869,-0.659839,1.796894,-0.568047,0.376008,-0.320880,-0.349202
and,3.032429,2.200934,0.168909,0.303117,-1.010507,0.456957,-0.254786,0.293911,1.154953,0.658438,...,-1.209527,-0.687960,-0.309287,-1.243712,-0.429132,0.672929,-0.768911,-0.385553,-0.153461,-0.164798
of,1.602295,2.747977,2.372136,-1.406231,-0.208143,-2.068553,0.424331,1.029849,0.685372,-1.977432,...,-0.838399,-0.960348,-1.479084,-0.708741,-0.829332,1.540681,-1.459929,0.072497,1.735249,0.346510
to,1.295550,0.404625,-0.774415,1.339003,-2.744981,-1.898481,0.406119,1.343802,1.319225,-1.747526,...,-0.852790,0.900618,0.937095,-3.117561,-1.944199,1.017031,-1.424121,0.299678,-1.781951,0.604400
is,0.586821,-0.538091,0.706562,-0.926518,0.182735,-0.774236,0.830147,-1.573713,1.324111,0.025818,...,0.102401,-0.089111,-0.628801,0.928474,-3.095457,4.187731,-0.315025,1.064841,-0.186651,1.746391


In [22]:
%%time
tokenized_array = np.array(tokenized)

model_array = np.array([word_vec_df.loc[doc].mean(axis=0) for doc in tokenized_array])

Wall time: 34.6 s


In [23]:
model_df = pd.DataFrame(model_array)
model_df["sentiment"] = df["sentiment"]

model_df.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,sentiment
0,0.891346,0.981726,-0.243343,0.200131,-0.288226,-0.396876,0.159949,0.344412,0.545034,-0.529693,...,-0.438629,-0.237874,0.219872,-0.225743,0.588679,-0.251512,0.229648,0.295378,0.045342,1.0
1,1.074119,0.718079,-0.121665,0.249437,-0.296867,-0.274530,0.363756,0.388845,0.469159,-0.545638,...,-0.281852,-0.546553,0.287645,-0.554801,0.506518,-0.502521,0.122082,0.312615,-0.026070,1.0
2,0.896156,0.854646,0.099905,-0.198713,-0.347461,-0.158461,0.164301,0.261809,0.371017,-0.184747,...,-0.237165,-0.385483,-0.353948,-0.321777,0.240935,-0.164179,0.073031,0.034520,-0.011251,0.0
3,0.944552,0.949661,0.050669,-0.138956,-0.336179,-0.413363,0.137519,0.332753,0.309960,-0.457349,...,-0.312368,-0.292819,0.051508,-0.354951,0.432797,-0.259300,0.183018,0.141218,0.002216,0.0
4,1.017126,0.922727,-0.071678,-0.164314,-0.403124,-0.228035,-0.015612,0.359387,0.430484,-0.130762,...,-0.447885,-0.255060,0.010728,-0.274360,0.276793,-0.130286,0.076927,0.067745,0.067427,1.0


In [24]:
model_df.shape
model_df.isnull().sum()

0                0
1                0
2                0
3                0
4                0
             ...  
252              0
253              0
254              0
255              0
sentiment    25000
Length: 257, dtype: int64

In [25]:
# 手动train_test_split
df_train=model_df[:25000]
df_test=model_df[25000:]
y=df_train["sentiment"]

In [26]:
del df_train["sentiment"]
text=df_train

In [27]:
# del df_test["sentiment"]
# test=df_test

In [28]:
text.shape, y.shape, test.shape

((25000, 256), (25000,), (25000, 256))

In [29]:
X_train, X_test, y_train, y_test = train_test_split(text, y, test_size=0.2,random_state=42)
y_train.values

array([0., 0., 0., ..., 1., 1., 0.])

# 模型预测

In [30]:
%%time
r.fit(X_train,y_train.values) # 随机森林
pred=r.predict(X_test)
pred = pred.astype(int)
pred

Wall time: 22.5 s


array([0, 1, 0, ..., 0, 0, 0])

In [31]:
print("classification_report")
print(classification_report(pred,y_test.values))

classification_report
              precision    recall  f1-score   support

           0       0.81      0.84      0.82      2395
           1       0.84      0.82      0.83      2605

    accuracy                           0.83      5000
   macro avg       0.83      0.83      0.83      5000
weighted avg       0.83      0.83      0.83      5000



In [32]:
from sklearn.model_selection import GridSearchCV
other_params = {'n_estimators': 10, 
                'bootstrap': True, 
                }

param_test1 = {
 'n_estimators':list(range(10,20, 30)),
}

r1 = RandomForestClassifier(**other_params)
gs1 = GridSearchCV(r1,param_test1,cv = 5,scoring = 'roc_auc',n_jobs = -1,verbose=2)
best_model1=gs1.fit(X_train, y_train.values.reshape(-1))
print('最优参数：',best_model1.best_params_)
print('最佳模型得分：',best_model1.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
最优参数： {'n_estimators': 10}
最佳模型得分： 0.8468700163965494


In [33]:
clr = RandomForestClassifier(n_estimators = 10)
r.fit(X_train,y_train.values) # 随机森林
pred=r.predict(X_test)
pred = pred.astype(int)
print("classification_report")
print(classification_report(pred,y_test.values))

classification_report
              precision    recall  f1-score   support

           0       0.81      0.84      0.82      2389
           1       0.85      0.82      0.83      2611

    accuracy                           0.83      5000
   macro avg       0.83      0.83      0.83      5000
weighted avg       0.83      0.83      0.83      5000



In [35]:
# sub.to_csv("submission_v1.csv", index = False)

In [36]:
%%time
LR = LogisticRegression(penalty = 'l2',
                       random_state = 42,
                       solver = 'sag',
                       max_iter = 100,
                       verbose = 0 )
LR.fit(X_train,y_train.values) # 逻辑斯蒂分类
pred=LR.predict(X_test)
pred = pred.astype(int)
print("classification_report")
print(classification_report(pred,y_test.values))

classification_report
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      2442
           1       0.89      0.88      0.88      2558

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000

Wall time: 3.37 s


In [38]:
other_params = {'penalty': 'l2',
               'random_state': 42,
               'solver': 'sag',
               'max_iter': 100,
               'verbose': 0 
                }

param_test1 = {
 'max_iter':list(range(100, 300, 500)),
}

lr1 = LogisticRegression(**other_params)
gs1 = GridSearchCV(lr1,param_test1,cv = 5,scoring = 'roc_auc',n_jobs = -1,verbose=2)
best_model1=gs1.fit(X_train, y_train.values.reshape(-1))
print('最优参数：',best_model1.best_params_)
print('最佳模型得分：',best_model1.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
最优参数： {'max_iter': 100}
最佳模型得分： 0.9454676537652643


In [41]:
sub=pd.DataFrame()
sub["id"]=test["id"]

pred=LR.predict(df_test)
pred = pred.astype(int)

sub['sentiment'] = pd.DataFrame(pred)
# sub.to_csv("submission_v2.csv", index = False)

In [42]:
# xgb = XGBClassifier()
other_params = {'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

param_test1 = {
 'max_depth':list(range(2,6,9)),
 'min_child_weight':list(range(2,4,6))
}

xgb = XGBClassifier(**other_params)
gs1 = GridSearchCV(xgb,param_test1,cv = 5,scoring = 'roc_auc',n_jobs = -1,verbose=2)
best_model1=gs1.fit(X_train, y_train.values.reshape(-1))
print('最优参数：',best_model1.best_params_)
print('最佳模型得分：',best_model1.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[20:02:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
最优参数： {'max_depth': 2, 'min_child_weight': 2}
最佳模型得分： 0.9060710836935246


In [43]:
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 2, 'min_child_weight': 2, 'seed': 0,
                    'subsample': 0.6, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
xgb = XGBClassifier(**other_params)
xgb.fit(X_train,y_train.values) # 随机森林
pred = xgb.predict(X_test)
pred = pred.astype(int)
print("classification_report")
print(classification_report(pred,y_test.values))

[20:02:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
classification_report
              precision    recall  f1-score   support

           0       0.85      0.87      0.86      2418
           1       0.88      0.86      0.87      2582

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [44]:
sub=pd.DataFrame()
sub["id"]=test["id"]

pred=xgb.predict(df_test)
pred = pred.astype(int)

sub['sentiment'] = pd.DataFrame(pred)
# sub.to_csv("submission_v3.csv", index = False)

* Random Forest Score: 0.82248
* Logistic Score: 0.87944
* XGBoost Score: 0.86352